In [6]:
import numpy as np

In [7]:
def objective_function(x):
    """Example objective function: Custom function with two variables."""
    return (x[0]-3.14)**2 + (x[1]-2.72)**2 + np.sin(3*x[0]+1.41) + np.sin(4*x[1]-1.73)

In [8]:
def grey_wolf_optimizer(obj_func, dim, bounds, max_iter, wolves_count):
    """
    Grey Wolf Optimizer (GWO) implementation.

    Parameters:
        obj_func (function): Objective function to minimize.
        dim (int): Number of dimensions.
        bounds (tuple): Lower and upper bounds for the search space as (lb, ub).
        max_iter (int): Maximum number of iterations.
        wolves_count (int): Number of wolves in the pack.

    Returns:
        best_position (numpy.ndarray): Best solution found.
        best_score (float): Objective value of the best solution.
    """
    # Initialize wolves' positions
    lb, ub = bounds
    wolves = np.random.uniform(lb, ub, (wolves_count, dim))

    # Initialize alpha, beta, delta positions and scores
    alpha_position = np.zeros(dim)
    beta_position = np.zeros(dim)
    delta_position = np.zeros(dim)
    alpha_score = np.inf  # Best score
    beta_score = np.inf   # Second-best score
    delta_score = np.inf  # Third-best score

    # Main optimization loop
    for t in range(max_iter):
        for i in range(wolves_count):
            fitness = obj_func(wolves[i])

            # Update alpha, beta, and delta
            if fitness < alpha_score:
                alpha_score, beta_score, delta_score = fitness, alpha_score, beta_score
                alpha_position, beta_position, delta_position = wolves[i], alpha_position, beta_position
            elif fitness < beta_score:
                beta_score, delta_score = fitness, beta_score
                beta_position, delta_position = wolves[i], beta_position
            elif fitness < delta_score:
                delta_score = fitness
                delta_position = wolves[i]

        # Update wolves' positions
        a = 2 - t * (2 / max_iter)  # Linearly decreasing a

        for i in range(wolves_count):
            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            A1 = 2 * a * r1 - a
            C1 = 2 * r2
            D_alpha = abs(C1 * alpha_position - wolves[i])
            X1 = alpha_position - A1 * D_alpha

            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            A2 = 2 * a * r1 - a
            C2 = 2 * r2
            D_beta = abs(C2 * beta_position - wolves[i])
            X2 = beta_position - A2 * D_beta

            r1, r2 = np.random.rand(dim), np.random.rand(dim)
            A3 = 2 * a * r1 - a
            C3 = 2 * r2
            D_delta = abs(C3 * delta_position - wolves[i])
            X3 = delta_position - A3 * D_delta

            wolves[i] = (X1 + X2 + X3) / 3

        # Ensure wolves stay within bounds
        wolves = np.clip(wolves, lb, ub)

    return alpha_position, alpha_score


In [15]:
# Example usage
dim = 2
bounds = (0, 5)
max_iter = 100
wolves_count = 20

best_position, best_score = grey_wolf_optimizer(objective_function, dim, bounds, max_iter, wolves_count)
print("Best position:", best_position)
print("Best score:", best_score)

Best position: [2.17674279 4.38814194]
Best score: -1.7908512066003859


In [16]:
# python implementation of Grey wolf optimization (GWO) 
# minimizing rastrigin and sphere function 
  
  
import random 
import math    # cos() for Rastrigin 
import copy    # array-copying convenience 
import sys     # max float 
  
  
#-------fitness functions--------- 
  
# rastrigin function 
def fitness_rastrigin(position): 
  fitness_value = 0.0
  for i in range(len(position)): 
    xi = position[i] 
    fitness_value += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return fitness_value 
  
#sphere function 
def fitness_sphere(position): 
    fitness_value = 0.0
    for i in range(len(position)): 
        xi = position[i] 
        fitness_value += (xi*xi); 
    return fitness_value; 
#------------------------- 
  
  
# wolf class  
class wolf: 
  def __init__(self, fitness, dim, minx, maxx, seed): 
    self.rnd = random.Random(seed) 
    self.position = [0.0 for i in range(dim)] 
  
    for i in range(dim): 
      self.position[i] = ((maxx - minx) * self.rnd.random() + minx) 
  
    self.fitness = fitness(self.position) # curr fitness 
  
  
  
# grey wolf optimization (GWO) 
def gwo(fitness, max_iter, n, dim, minx, maxx): 
    rnd = random.Random(0) 
  
    # create n random wolves  
    population = [ wolf(fitness, dim, minx, maxx, i) for i in range(n)] 
  
    # On the basis of fitness values of wolves  
    # sort the population in asc order 
    population = sorted(population, key = lambda temp: temp.fitness) 
  
    # best 3 solutions will be called as  
    # alpha, beta and gaama 
    alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[: 3]) 
  
  
    # main loop of gwo 
    Iter = 0
    while Iter < max_iter: 
  
        # after every 10 iterations  
        # print iteration number and best fitness value so far 
        if Iter % 10 == 0 and Iter > 1: 
            print("Iter = " + str(Iter) + " best fitness = %.3f" % alpha_wolf.fitness) 
  
        # linearly decreased from 2 to 0 
        a = 2*(1 - Iter/max_iter) 
  
        # updating each population member with the help of best three members  
        for i in range(n): 
            A1, A2, A3 = a * (2 * rnd.random() - 1), a * ( 
              2 * rnd.random() - 1), a * (2 * rnd.random() - 1) 
            C1, C2, C3 = 2 * rnd.random(), 2*rnd.random(), 2*rnd.random() 
  
            X1 = [0.0 for i in range(dim)] 
            X2 = [0.0 for i in range(dim)] 
            X3 = [0.0 for i in range(dim)] 
            Xnew = [0.0 for i in range(dim)] 
            for j in range(dim): 
                X1[j] = alpha_wolf.position[j] - A1 * abs( 
                  C1 * alpha_wolf.position[j] - population[i].position[j]) 
                X2[j] = beta_wolf.position[j] - A2 * abs( 
                  C2 *  beta_wolf.position[j] - population[i].position[j]) 
                X3[j] = gamma_wolf.position[j] - A3 * abs( 
                  C3 * gamma_wolf.position[j] - population[i].position[j]) 
                Xnew[j]+= X1[j] + X2[j] + X3[j] 
              
            for j in range(dim): 
                Xnew[j]/=3.0
              
            # fitness calculation of new solution 
            fnew = fitness(Xnew) 
  
            # greedy selection 
            if fnew < population[i].fitness: 
                population[i].position = Xnew 
                population[i].fitness = fnew 
                  
        # On the basis of fitness values of wolves  
        # sort the population in asc order 
        population = sorted(population, key = lambda temp: temp.fitness) 
  
        # best 3 solutions will be called as  
        # alpha, beta and gaama 
        alpha_wolf, beta_wolf, gamma_wolf = copy.copy(population[: 3]) 
          
        Iter+= 1
    # end-while 
  
    # returning the best solution 
    return alpha_wolf.position 
            
#---------------------------- 
  
  
# Driver code for rastrigin function 
  
print("\nBegin grey wolf optimization on rastrigin function\n") 
dim = 3
fitness = fitness_rastrigin 
  
  
print("Goal is to minimize Rastrigin's function in " + str(dim) + " variables") 
print("Function has known min = 0.0 at (", end="") 
for i in range(dim-1): 
  print("0, ", end="") 
print("0)") 
  
num_particles = 50
max_iter = 100
  
print("Setting num_particles = " + str(num_particles)) 
print("Setting max_iter    = " + str(max_iter)) 
print("\nStarting GWO algorithm\n") 
  
  
  
best_position = gwo(fitness, max_iter, num_particles, dim, -10.0, 10.0) 
  
print("\nGWO completed\n") 
print("\nBest solution found:") 
print(["%.6f"%best_position[k] for k in range(dim)]) 
err = fitness(best_position) 
print("fitness of best solution = %.6f" % err) 
  
print("\nEnd GWO for rastrigin\n") 
  
  
print() 
print() 
  
  
# Driver code for Sphere function  
print("\nBegin grey wolf optimization on sphere function\n") 
dim = 3
fitness = fitness_sphere 
  
  
print("Goal is to minimize sphere function in " + str(dim) + " variables") 
print("Function has known min = 0.0 at (", end="") 
for i in range(dim-1): 
  print("0, ", end="") 
print("0)") 
  
num_particles = 50
max_iter = 100
  
print("Setting num_particles = " + str(num_particles)) 
print("Setting max_iter    = " + str(max_iter)) 
print("\nStarting GWO algorithm\n") 
  
  
  
best_position = gwo(fitness, max_iter, num_particles, dim, -10.0, 10.0) 
  
print("\nGWO completed\n") 
print("\nBest solution found:") 
print(["%.6f"%best_position[k] for k in range(dim)]) 
err = fitness(best_position) 
print("fitness of best solution = %.6f" % err) 
  
print("\nEnd GWO for sphere\n") 


Begin grey wolf optimization on rastrigin function

Goal is to minimize Rastrigin's function in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting GWO algorithm

Iter = 10 best fitness = 6.636
Iter = 20 best fitness = 1.047
Iter = 30 best fitness = 1.012
Iter = 40 best fitness = 1.010
Iter = 50 best fitness = 1.008
Iter = 60 best fitness = 1.008
Iter = 70 best fitness = 1.008
Iter = 80 best fitness = 1.006
Iter = 90 best fitness = 1.005

GWO completed


Best solution found:
['-0.004395', '0.995042', '0.005800']
fitness of best solution = 1.005465

End GWO for rastrigin




Begin grey wolf optimization on sphere function

Goal is to minimize sphere function in 3 variables
Function has known min = 0.0 at (0, 0, 0)
Setting num_particles = 50
Setting max_iter    = 100

Starting GWO algorithm

Iter = 10 best fitness = 0.000
Iter = 20 best fitness = 0.000
Iter = 30 best fitness = 0.000
Iter = 40 best fitness = 0.000
Iter = 50